# Data cleaning and preprocessing

## imports 

In [2]:
import random
import datetime

import pandas as pd 
import numpy as np 

# round to two decimal places in python pandas
pd.options.display.float_format = '{:.2f}'.format

random.seed(999)

## Loading  and checking data 

In [3]:
# Checking size and first rows of files train and test 
row_numbers_train = sum(1 for row in open('../data/train.csv'))
row_numbers_test = sum(1 for row in open('../data/test.csv'))

total_rows = row_numbers_train + row_numbers_test

print(f'train.csv file has: {row_numbers_train} rows ({round(row_numbers_train/total_rows,4)*100}%)')
print(f'test.csv file has: {row_numbers_test} rows ({round(row_numbers_test/total_rows,4)*100}%)')

train.csv file has: 55423857 rows (99.98%)
test.csv file has: 9915 rows (0.02%)


To avoid some memory problems loading all 55 MM of files lets make a sample in train file to train models. We also 'll leave Test.csv to final validation. we 'll use a "validation sample" as our test dataset.

In [4]:
# defining a util function 
def get_sample_pandas(
        sample_size:int,
        population_filepath:str=None,
        file_format:str='csv',
        head:bool= True
)-> pd.DataFrame:
    """function to receive a sample size and bootsrap using random a number or rows
        loaded in pandas.
    """
    if head is True:
        start_step = 1 
    else:
        start_step = 0
    #row count on population files    
    population_rows = sum (1 for row in open(population_filepath))
    
    #bootstap of rows to not be skipped
    skipped_rows = sorted(
        random.sample(
            range(start_step,population_rows+1),
            population_rows - sample_size
        )
    )

    return pd.read_csv(population_filepath,skiprows=skipped_rows)

Lets start with 5.5 millions of rows to explore and model data

In [38]:
sample_size = 5_500_000

In [5]:
dataset = get_sample_pandas(sample_size= sample_size,population_filepath='../data/train.csv')
dataset.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2011-08-18 00:35:00.00000049,5.70,2011-08-18 00:35:00 UTC,-73.98,40.76,-73.99,40.75,2
1,2013-11-23 12:57:00.000000190,5.00,2013-11-23 12:57:00 UTC,0.00,0.00,0.00,0.00,1
2,2012-05-04 06:11:20.0000002,9.30,2012-05-04 06:11:20 UTC,-73.99,40.69,-74.00,40.73,1
3,2011-03-29 15:39:49.0000002,8.10,2011-03-29 15:39:49 UTC,-73.98,40.74,-73.98,40.76,1
4,2014-05-01 09:12:00.000000198,7.00,2014-05-01 09:12:00 UTC,-73.97,40.77,-73.98,40.77,6


In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5500000 entries, 0 to 5499999
Data columns (total 8 columns):
 #   Column             Dtype  
---  ------             -----  
 0   key                object 
 1   fare_amount        float64
 2   pickup_datetime    object 
 3   pickup_longitude   float64
 4   pickup_latitude    float64
 5   dropoff_longitude  float64
 6   dropoff_latitude   float64
 7   passenger_count    int64  
dtypes: float64(5), int64(1), object(2)
memory usage: 335.7+ MB


 observations:

- Some values is null.
- Pickup_datetime values its look's like datetime format but it is formated as object. it is posible to change that to datatime (key is designed to be a ID_event and not interpreted as date)
- Some variables coud be downcasted to lower type formats reduce and optimize calculations. 
- target variable is fare_amout

In [9]:

# getting cols with nulls
null_cols = [ col for col in dataset.columns if dataset[col].isnull().sum()>0]

#print 2 rows of each colummn to  sanity check
for column in null_cols:
    print(f" Column :  |{column}|")
    print("="*80)
    print(dataset[dataset[column].isnull()].head(1))
    print("="*80)

 Column :  |dropoff_longitude|
                               key  fare_amount          pickup_datetime  \
79226  2013-11-04 20:07:59.0000006         7.20  2013-11-04 20:07:59 UTC   

       pickup_longitude  pickup_latitude  dropoff_longitude  dropoff_latitude  \
79226            -73.98            40.79                NaN               NaN   

       passenger_count  
79226                0  
 Column :  |dropoff_latitude|
                               key  fare_amount          pickup_datetime  \
79226  2013-11-04 20:07:59.0000006         7.20  2013-11-04 20:07:59 UTC   

       pickup_longitude  pickup_latitude  dropoff_longitude  dropoff_latitude  \
79226            -73.98            40.79                NaN               NaN   

       passenger_count  
79226                0  


in this case passengers_count = 0 also looks like a error... 

In [10]:
# according to https://en.wikipedia.org/wiki/Decimal_degrees we can have a precision of tectonic survey using 
# only 8 decimal places is needed. latitute and longitude could be downcasted to a format that handles 8 decimals
#8 bits coud represent a number of 256 digits... that's makes enough to a taxi fare... 
new_types = {
    'fare_amount':np.float32, #   'pickup_datetime': datetime.datetime, 
    'pickup_longitude':np.float32,
    'pickup_latitude':np.float32,
    'dropoff_longitude':np.float32,	
    'dropoff_latitude':np.float32,
    'passenger_count':np.int8
}

dataset=dataset.astype(new_types)
# date to datetime 
dataset.pickup_datetime = pd.to_datetime(dataset.pickup_datetime)

In [13]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5500000 entries, 0 to 5499999
Data columns (total 8 columns):
 #   Column             Dtype              
---  ------             -----              
 0   key                object             
 1   fare_amount        float32            
 2   pickup_datetime    datetime64[ns, UTC]
 3   pickup_longitude   float32            
 4   pickup_latitude    float32            
 5   dropoff_longitude  float32            
 6   dropoff_latitude   float32            
 7   passenger_count    int8               
dtypes: datetime64[ns, UTC](1), float32(5), int8(1), object(1)
memory usage: 194.1+ MB


We reduced more than 50% of memory usage on dataframe

In [14]:
#before remove nan
dataset.isnull().sum()

key                   0
fare_amount           0
pickup_datetime       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude    33
dropoff_latitude     33
passenger_count       0
dtype: int64

The maximum amount of passengers allowed in a yellow taxicab by law is four (4) in a four (4) passenger taxicab 
or five (5) passengers in a five (5) passenger taxicab,

source :https://www.nyc.gov/site/tlc/passengers/passenger-frequently-asked-questions.page


## Cleaning some sanity checks

In [17]:
# delete all rows with zero passenger_count
index_passengers = dataset[ (dataset['passenger_count'] <= 0) | (dataset['passenger_count'] >5)  ].index
dataset.drop(index_passengers , inplace=True)


In [23]:
# delete all rows with zero or negative fare amount
index_amout = dataset[ (dataset['fare_amount'] <= 0)].index
dataset.drop(index_amout , inplace=True)

In [25]:
# delete rows with zero coordenates 
index_pickup = dataset[ 
    (dataset['pickup_longitude'] == 0) |
    (dataset['pickup_latitude'] == 0) |
    (dataset['dropoff_longitude'] == 0) |
    (dataset['dropoff_latitude'] == 0) 
].index
dataset.drop(index_pickup , inplace=True)

In [26]:
# since we have a lot data lets assume that 'll not be prejudicial drop na from sample
dataset.dropna(inplace=True)

In [27]:
#after remove nan amd other errors
dataset.isnull().sum()

key                  0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64

In [40]:
deleted_rows = sample_size - dataset.shape[0]
print(f' it was deleted {deleted_rows} ({round(deleted_rows/sample_size*100)}%) from dataframe after sanity checks')

 it was deleted 243230 (4%) from dataframe after sanity checks


checking distribution of data. Included 0.01 and 0.999 because, thats problably 'll indicate some outliers 

In [41]:
#describing 
dataset.describe([.01,.05,.10,.25,.5,.75,.9,.99,.999])

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,5256770.00,5256770.00,5256770.00,5256770.00,5256770.00,5256770.00
mean,11.32,-72.09,42.92,-72.09,42.91,1.60
std,42.01,7.57,7.87,8.54,7.43,1.16
min,0.01,-3414.13,-3492.26,-3428.04,-3461.54,1.00
1%,3.30,-74.01,40.65,-74.02,40.65,1.00
5%,4.10,-74.01,40.71,-74.01,40.70,1.00
10%,4.50,-74.00,40.72,-74.00,40.72,1.00
25%,6.00,-73.99,40.74,-73.99,40.74,1.00
50%,8.50,-73.98,40.75,-73.98,40.75,1.00
75%,12.50,-73.97,40.77,-73.97,40.77,2.00


## Exporting preprocessed dataframe to csv file

In [42]:
# dropping key column
dataset.drop(columns='key',inplace=True)

In [43]:
dataset.to_csv(f'../data/taxi_sample.csv')